In [1]:
%pylab nbagg

Populating the interactive namespace from numpy and matplotlib


In [8]:
import numpy as np
import os
import pyopencl as cl
import pyopencl.array as cla

# Create context and queue
ctx = cl.create_some_context(interactive=True)
queue = cl.CommandQueue(ctx)


Choose platform:
[0] <pyopencl.Platform 'Intel Gen OCL Driver' at 0x7fadbcea8740>
[1] <pyopencl.Platform 'Intel(R) OpenCL' at 0x557f37cd0610>
[2] <pyopencl.Platform 'AMD Accelerated Parallel Processing' at 0x7fadb5145430>
Choice [0]:
Set the environment variable PYOPENCL_CTX='' to avoid being asked again.


In [9]:
# Create host and device arrays
N = 10000
a = np.arange(N, dtype=np.float32)
b = np.ones((N,), dtype=np.float32)
d_a = cla.to_device(queue, a)
d_b = cla.to_device(queue, b)
d_res = cla.empty_like(d_a)

In [10]:
%load_ext pyopencl.ipython_ext

#load magic expression to code kernel in notebook

The pyopencl.ipython_ext extension is already loaded. To reload it, use:
  %reload_ext pyopencl.ipython_ext


In [11]:
%%cl_kernel 

//define the kernel:
    
kernel void gpu_add(
        global float* arr1, 
        global float* arr2, 
        global float* res, 
        int N) 
    {
        int tid = get_global_id(0); // Thread ID
        if (tid < N) {
            res[tid] = arr1[tid] + arr2[tid];
        }
     }

In [12]:
# Build and execute the kernel
gpu_add(queue, (N,), None, 
        d_a.data, d_b.data, d_res.data, np.int32(N))
# retrieve the result from GPU
numpy.allclose(a+b, d_res.get())

True